<a href="https://colab.research.google.com/github/raheelam98/LangGraph/blob/main/fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lang Chain and Lang Graph Fundaments

In [ ]:
# install the required packages
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_google_genai

In [ ]:
# API Keys
import os
from google.colab import userdata

# os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "quickstart"

gemini_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

In [ ]:
# check llm is working
result = llm.invoke("hi")
result

**Tutorials**

[How-to Guides - LangGraph](https://langchain-ai.github.io/langgraph/how-tos/)


### Module 1

[Module 1 - GitHub](https://github.com/panaversity/learn-applied-generative-ai-fundamentals/tree/main/03_langchain_ecosystem/langgraph/course-notebooks/module-1)

[Q3,Q4,Q5 - Class -14: LangGraph - Six steps to create a graph : Agentic Architecture - Oct 17, 2024](https://www.youtube.com/watch?v=7U_9hCE3Prs)

**3_router.ipynb**

**`Tool calling`**: Utilizes a chat model API that handles tool schemas and messages, producing tool invocations as output.

**`ToolNode`** Usage: ToolNode, a LangChain Runnable, processes graph state (with messages) and updates state with tool call results, seamlessly integrating with LangGraph's ReAct agent or any StateGraph with MessagesState.

ToolNode: Takes a list of functions and stores them within the tool node.

When any value is stored in it, ToolNode makes it runnable, calls it to perform the functionality, and provides the response

ToolNode == Runable

```bash
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode

# Build graph
builder: StateGraph = StateGraph(MessagesState)

graph_builder.add_node("tools", ToolNode([multiply]))
```